<a href="https://colab.research.google.com/github/johnnytorresm/stackoverflow/blob/main/fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Installing FastText

# Cloning fastText from GitHub
! git clone https://github.com/facebookresearch/fastText.git

# import os library
import os
 
# change the current directory to specified directory
os.chdir(r"./fastText")
 
print("Directory changed")

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 34.52 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.


In [84]:
! ls -l

total 28476
drwxr-xr-x 2 root root     4096 Sep 25 18:01 alignment
-rw-r--r-- 1 root root    80592 Sep 25 21:10 args.o
-rw-r--r-- 1 root root   104976 Sep 25 21:10 autotune.o
drwxr-xr-x 5 root root     4096 Sep 25 18:38 build
-rwxr-xr-x 1 root root     1428 Sep 25 18:01 classification-example.sh
-rwxr-xr-x 1 root root     3154 Sep 25 18:01 classification-results.sh
-rw-r--r-- 1 root root     2214 Sep 25 18:01 CMakeLists.txt
-rw-r--r-- 1 root root     3356 Sep 25 18:01 CODE_OF_CONDUCT.md
-rw-r--r-- 1 root root     2061 Sep 25 18:01 CONTRIBUTING.md
-rw-r--r-- 1 root root  1859871 Sep 25 21:55 corpus10k.bin
-rw-r--r-- 1 root root 12393293 Sep 25 18:42 corpus10k.csv
-rw-r--r-- 1 root root        0 Sep 25 21:55 corpus10k.result
-rw-r--r-- 1 root root   700856 Sep 25 21:34 corpus10k.test
-rw-r--r-- 1 root root  6300834 Sep 25 21:34 corpus10k.train
-rw-r--r-- 1 root root  3033081 Sep 25 21:55 corpus10k.vec
drwxr-xr-x 2 root root     4096 Sep 25 18:01 crawl
drwxr-xr-x 3 root root     4096 Sep 

In [33]:
#-----------------------------
# Importing libraries
#-----------------------------
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime
import glob
import string 
import os
import io
import subprocess
from scipy import sparse
import csv
import codecs

import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

from datetime import timedelta
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import fastText as ft

from collections import Counter

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

import sklearn.metrics as metrics

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

from ipywidgets import *
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

import ast
from collections import Counter

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

print('\nLibraries have been imported')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

Libraries have been imported


In [34]:
#--------------------
# Global Variables
#--------------------
sw  = set(stopwords.words())

# Exhaustive list of English prepositions to be added to stop words
preps = pd.read_csv('other-stop-words.txt', header=None, names=['other-stop-words'])
preps_set = set(preps['other-stop-words'].tolist())
sw |= preps_set

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

print('Global Variables have been defined')


Global Variables have been defined


In [35]:
#-----------------------------------------
# Function definitions
#-----------------------------------------

#----------------------------------------------------
# clean_txt: Cleans a string, and then lemmatize it.
#----------------------------------------------------
def clean_txt(txt): 
    # txt: string to clean
    
    global sw, lemmatizer                # sw = English stop words; lemmatizer from WordNet

    # 1. Lowercase 
    txt = txt.lower()

    # 2. Strip HTML code
    txt = re.sub('<[^<]+?>', ' ', txt)

    # 3. Strip punctuation signs
    txt = re.sub('[^\w\s]', ' ', txt)
    txt = re.sub('\*', ' ', txt)

    # 4. remove stop words
    txt_set = set(nltk.word_tokenize(txt))
    txt_lst = [w for w in txt_set if (not w in sw)]
    txt_set = set(txt_lst)
    txt_lst = list(txt_set)
    
    # 5. Lemmatisation
    txt_lst = [lemmatizer.lemmatize(w, pos='v') for w in txt_lst]
    txt = ' '.join([lemmatizer.lemmatize(w, pos='n') for w in txt_lst])
    
    return txt

#-----------------------------------------
# display_topics after LDA
#-----------------------------------------
def display_topics(model, feature_names, no_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        topic_list = " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words-1:-1]])
        print(topic_list)
        topics.append(topic_list)
    return topics
    
#-----------------------------------------
# Print a classifier's scores
#-----------------------------------------
def print_score(y_true, y_pred, clf):
    print("Classifier: ", clf.__class__.__name__)
    print("Precision score : {}".format(precision_score(y_true, y_pred, average='weighted')))
    print("Recall score : {}".format(recall_score(y_true, y_pred, average='weighted')))
    print("F1 score : {}".format(f1_score(y_true, y_pred, average='weighted')))

#-----------------------------------------
# Print fasttext's scores
#-----------------------------------------
def ft_results(N, p, r):
    print("N\t" + str(N))
    f1 = 2 * ( p * r ) / ( p + r)   # calculate f1 score
    print("P@{}\t{:.4f}".format(1, p))
    print("R@{}\t{:.4f}".format(1, r))
    print("F1@{}\t{:.4f}".format(1, f1))
    
#-----------------------------------------
# Receives a list of tags and returns
# a list of Labels in fasttext format
#-----------------------------------------
def labels(Tags):
    y_labels = ' '
    for t in Tags:
        y_labels += '__label__'+ t + ' '
    
    return y_labels

#-----------------------------------------
# most frequent tags in the corpus
#-----------------------------------------
def most_used_tags(tags, top_tags):
    final_tags = []
    for tag in tags:
        if tag in top_tags['Tags'].values:
            final_tags.append(tag)
    return final_tags

#-----------------------------------------
# Select file from a directory
#-----------------------------------------
def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True) # List of selected files will be set button's file attribute.
    print(b.files) # Print the list of files selected.
    
#-----------------------------------------
# Remove "__label__" from FastText Labels
#-----------------------------------------
def remove_label(l,prefix)-> str:
    if l.startswith(prefix):
        return l[len(prefix):]
    else:
        return l[:]

#---------------------------------------------
# Sorts a dictionary by value. Returns a list.
#---------------------------------------------
def sort_tags(x):
    if len(x)>0:
        s = sorted(x.items(), key = lambda y:(y[1], y[0]), reverse = True)
    else:
        s = []
    return s

#---------------------------------------------
# Top 5 values 
#---------------------------------------------
def top_tags(x):
    s = []
    i = 0
    if len(x)>0:
        while ((i<len(x)) and (i<5)):
            s = s + [x[i][0]]
            i += 1
    else:
        s = []
    
    return s

print('Functions have been defined')


Functions have been defined


In [36]:
#--------------------------------
# Reading data
#--------------------------------

file2open = "corpus10k.csv"

print('Reading...')
posts = pd.read_csv(file2open, usecols=['Id', 'Tags', 'Text'])

# Number of records read
display(posts.shape)
display(posts.head())
        
# counting nulls per column
display(posts.isnull().sum())


Reading...


(11146, 3)

,Id,Tags,Text
0,4,"['c#', 'floating-point', 'type-conversion', 'd...",change error vb use work 5000 track try contro...
1,6,"['html', 'css', 'internet-explorer-7']",collapse miss explorer 0 children specificatio...
2,9,"['c#', '.net', 'datetime']",someone datetime represent base age calculate ...
3,11,"['c#', 'datetime', 'time', 'datediff', 'relati...",display datetime days 2 time month 3 specific ...
4,13,"['html', 'browser', 'timezone', 'user-agent', ...",timezone http part agent determine user string...


Id      0
Tags    0
Text    0
dtype: int64

In [38]:
#----------------------------------------------------------------
# Checking if Tags have been read as a single string or a series
#-----------------------------------------------------------------
if isinstance(posts['Tags'].iloc[0],str): # verifies if 'Tags' is of type "string"
    display(posts['Tags'].iloc[0])
    posts['Tags'] = posts['Tags'].apply(lambda tag: ast.literal_eval(tag))
    display(posts['Tags'].iloc[0])

print('Ready')


Ready


In [39]:
#------------------------------------------------------
# Flatten the "Tags" columns into a Series and then 
# count the number of occurrences per item, putting
# the results into a dataframe
#------------------------------------------------------
tag_series = pd.Series([item for sublist in posts['Tags'] for item in sublist])
tag_df = tag_series.groupby(tag_series).size().rename_axis('Tags').reset_index(name='Nº of occurrences')
tag_df = tag_df.sort_values(by=['Nº of occurrences'], ascending=False)
tag_df.head(20)

# 2
# Nº of Tags that appear more than 100 times across all messages

series = tag_df['Nº of occurrences'].apply(lambda x: True if x > 10 else False)
counting = len(series[series == True].index)

print('Nº of Tags that appear more than 10 times across all messages:', counting)


Nº of Tags that appear more than 10 times across all messages: 454


In [41]:
#-----------------------------------------
# Top 100 tags
#-----------------------------------------
top_tags = tag_df[['Tags', 'Nº of occurrences']].head(100)
display(top_tags)

#--------------------------
# y = list of Tags list
#--------------------------
posts['Tags'] = posts['Tags'].apply(lambda tags: most_used_tags(tags, top_tags))
posts = posts.loc[posts['Tags'].str.len() > 0]
y = posts['Tags']
print('Ready - Nº of chosen tags', len(y))

#--------------------
# Display results
#--------------------
display(posts.head()), display(y)


,Tags,Nº of occurrences
407,c#,1370
2,.net,1237
1777,java,807
195,asp.net,693
410,c++,527
...,...,...
3760,vba,50
839,date,49
1639,image,49
811,database-design,49


Ready - Nº of chosen tags 9591


,Id,Tags,Text
0,4,[c#],change error vb use work 5000 track try contro...
1,6,"[html, css]",collapse miss explorer 0 children specificatio...
2,9,"[c#, .net]",someone datetime represent base age calculate ...
3,11,[c#],display datetime days 2 time month 3 specific ...
4,13,"[html, browser]",timezone http part agent determine user string...


0                               [c#]
1                        [html, css]
2                         [c#, .net]
3                               [c#]
4                    [html, browser]
                    ...             
11139          [c++, multithreading]
11140                       [vb.net]
11141    [.net, asp.net, javascript]
11142              [javascript, oop]
11145                     [security]
Name: Tags, Length: 9591, dtype: object

(None, None)

In [42]:
#------------------------------------
# Apply MultiLabelBinarizer to Tags
#------------------------------------
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(posts['Tags'])
y[0:10,0:10]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [43]:
#------------
# TD-IDF 
#------------
tfidf = TfidfVectorizer(analyzer="word", max_features=1000, ngram_range=(1,1))
X = tfidf.fit_transform(posts['Text'])

print('Ready')


Ready


In [44]:
#------------------------------
# Split into train and test
#------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape,  X_test.shape,  y_train.shape, y_test.shape


((7672, 1000), (1919, 1000), (7672, 100), (1919, 100))

In [45]:
#----------------------------------
# Multilabel with LinearSVC()
#----------------------------------

print('Multilabel with LinearSVC() - Begin..', datetime.datetime.now())

# Create the SVM objetc
svm = LinearSVC(random_state=42)

# Make it an Multilabel classifier
multilabel_classifier = MultiOutputClassifier(svm, n_jobs=-1)

# Fit the data to the Multilabel classifier
multilabel_classifier = multilabel_classifier.fit(X_train, y_train)

# Get predictions for test data
y_pred = multilabel_classifier.predict(X_test)

print_score(y_test, y_pred, svm)

print('Multilabel with LinearSVC() - End..', datetime.datetime.now())


Multilabel with LinearSVC() - Begin.. 2021-09-25 19:54:41.337058
Classifier:  LinearSVC
Precision score : 0.6461091951271151
Recall score : 0.36437859086785607
F1 score : 0.4434978836056442
Multilabel with LinearSVC() - End.. 2021-09-25 19:54:44.156257


In [47]:
#----------------------------------------------
# Multilabel with MultinomialNB()
#----------------------------------------------
# Create the SVM objet
svm2 = MultinomialNB()

print('Multinomial NB - Begin..', datetime.datetime.now())

# Make it an Multilabel classifier
multilabel_classifier2 = MultiOutputClassifier(svm2, n_jobs=-1)

# Fit the data to the Multilabel classifier
multilabel_classifier2 = multilabel_classifier2.fit(X_train, y_train)

# Get predictions for test data
y_pred2 = multilabel_classifier2.predict(X_test)

print_score(y_test, y_pred2, svm2)

print('Multinomial NB - End..', datetime.datetime.now())


Multinomial NB - Begin.. 2021-09-25 19:56:11.993135
Classifier:  MultinomialNB
Precision score : 0.2010634008668481
Recall score : 0.005745388569700635
F1 score : 0.011079655160598062
Multinomial NB - End.. 2021-09-25 19:56:12.820765


In [48]:
#---------------------------------------
# Multilabel with SGDClassifier()
#----------------------------------------

# Create the SVM objet
svm3 = SGDClassifier()

print('Multinlabel with SGDClassifier - Begin..', datetime.datetime.now())

# Make it an Multilabel classifier
multilabel_classifier3 = MultiOutputClassifier(svm3, n_jobs=-1)

# Fit the data to the Multilabel classifier
multilabel_classifier3 = multilabel_classifier3.fit(X_train, y_train)

# Get predictions for test data
y_pred3 = multilabel_classifier3.predict(X_test)

print_score(y_test, y_pred3, svm3)

print('Multinlabel with SGDClassifier - End..', datetime.datetime.now())


Multinlabel with SGDClassifier - Begin.. 2021-09-25 19:58:08.028203
Classifier:  SGDClassifier
Precision score : 0.6650479977518241
Recall score : 0.34260659207741156
F1 score : 0.41156738064177895
Multinlabel with SGDClassifier - End.. 2021-09-25 19:58:09.388413


In [49]:
#-----------------------------------------------------
# Multilabel with LogisticRegression (solver='saga')
#-----------------------------------------------------

# Create the SVM objet
svm4 = LogisticRegression(solver='saga')

print('Multilabel with LogisticRegression - Begin..', datetime.datetime.now())

# Make it an Multilabel classifier
multilabel_classifier4 = MultiOutputClassifier(svm4, n_jobs=-1)

# Fit the data to the Multilabel classifier
multilabel_classifier4 = multilabel_classifier4.fit(X_train, y_train)

# Get predictions for test data
y_pred4 = multilabel_classifier4.predict(X_test)
print_score(y_test, y_pred4, svm4)

print('Multilabel with LogisticRegression - End..', datetime.datetime.now())


Multilabel with LogisticRegression - Begin.. 2021-09-25 20:02:14.089220
Classifier:  LogisticRegression
Precision score : 0.5843260788581623
Recall score : 0.15240399153311157
F1 score : 0.22618241640036535
Multilabel with LogisticRegression - End.. 2021-09-25 20:02:42.438623


In [50]:
#-------------------------------------------------------
# GridSearch for RandomForestClassifier
#-------------------------------------------------------

print('Begin... RandomForestClassifier', datetime.datetime.now())

parameters = {'n_estimators'      : [100, 200],
              'max_depth'         : [20, 30],
              'random_state'      : [0],
              'verbose'           : [1],
              'max_features'      : ['auto'],
              'criterion'         : ['gini']
             }

clf = RandomizedSearchCV(RandomForestClassifier(), parameters, cv=5, n_jobs=-1)
clf.fit(X_train, y_train)

print(clf.score(X_train, y_train))
print(clf.best_params_)

# Testing RandomForestClassifier with the best params found above
print('Testing RandomForestClassifier with the best params found above...')

rfc = RandomForestClassifier(n_estimators = clf.best_params_['n_estimators'],
                             max_depth = clf.best_params_['max_depth'],
                             random_state = 0,
                             verbose = 0,
                             max_features = 'auto',
                             criterion = 'gini'
                            )
rfc.fit(X_train, y_train)

y_pred6 = rfc.predict(X_test)
print_score(y_test, y_pred6, rfc)

print('End...', datetime.datetime.now())


Begin... RandomForestClassifier 2021-09-25 20:09:14.370810


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   29.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    6.3s finished


0.21037539103232533
{'verbose': 1, 'random_state': 0, 'n_estimators': 100, 'max_features': 'auto', 'max_depth': 30, 'criterion': 'gini'}
Testing RandomForestClassifier with the best params found above...
Classifier:  RandomForestClassifier
Precision score : 0.3674324405382363
Recall score : 0.11339582703356517
F1 score : 0.151070418197909
End... 2021-09-25 20:19:32.083393


In [51]:
#-------------------------------------------------------
# Preparation of tags for fasttext 
# The string "__label__" is attached to each tag to create a new column named 'Labels'.
# Columns 'Labels' and 'Text' are then written to an external txt file for fasttext analysis (in OS mode)
#-------------------------------------------------------
train, test = train_test_split(posts, test_size=0.2)

train['Labels'] = train['Tags'].apply(lambda x: labels(x))
test['Labels'] = test['Tags'].apply(lambda x: labels(x))

train = train.dropna()
test = test.dropna()

display(train[['Tags', 'Labels', 'Text']].head(10))
display(test[['Tags', 'Labels', 'Text']].head(10))


,Tags,Labels,Text
3603,"[jquery, ajax, http]",__label__jquery __label__ajax __label__http,response copy copy http object sort status ret...
4632,"[javascript, html]",__label__javascript __label__html,solution textarea length gt enter cross number...
11041,"[windows, winapi]",__label__windows __label__winapi,create controll auto arrange place look contro...
4667,"[database, oracle]",__label__database __label__oracle,another sid service oracle different ora need ...
5323,[actionscript-3],__label__actionscript-3,ecmascript native 0 actionscript value result ...
1630,"[visual-studio-2008, eclipse]",__label__visual-studio-2008 __label__eclipse,f11 configure info f5 need use easy keyboard m...
11051,[java],__label__java,suggestions mavenproject hint object plugin ea...
2683,[javascript],__label__javascript,object property undefined check javascript det...
8920,"[jquery, ajax]",__label__jquery __label__ajax,error come parse http submit post unable make ...
7459,[algorithm],__label__algorithm,content automatically text categories categori...


,Tags,Labels,Text
9230,"[sql, database]",__label__sql __label__database,error specify obviously check select foo spell...
5107,[linux],__label__linux,error find quick file miss directory home inst...
8093,[eclipse],__label__eclipse,annoy usually date finally file menu branch th...
7625,"[java, multithreading]",__label__java __label__multithreading,various http different seem run specification ...
2157,"[c#, asp.net, authentication]",__label__c# __label__asp.net __label__authent...,solution hard info unable save automate figure...
8329,[user-interface],__label__user-interface,create file image use button photoshop friend ...
2823,[python],__label__python,tool small search reference live replace admin...
7716,[java],__label__java,create necessary get single dependency magic m...
7884,"[sql, oracle]",__label__sql __label__oracle,store tool options procedures oracle write use...
9830,"[c++, windows, command-line]",__label__c++ __label__windows __label__comman...,hope various file tool say fire status develop...


In [71]:
#-------------------------------------------------------
# It writes Labels and Text in FastText format to train.txt and test.txt files
# Those files will be the input for FastText routines
#-------------------------------------------------------

train_file = os.path.splitext(file2open)[0] + '.train'
test_file  = os.path.splitext(file2open)[0] + '.test'

train.to_csv(train_file, 
             header=None, 
             index=None, 
             mode='a', 
             encoding='utf-8', 
             columns=['Labels', 'Text'],
             sep=' ',
             escapechar=' ',
             quoting=csv.QUOTE_NONE,
             quotechar=' '
            )

print('Ready ', train_file)

test.to_csv(test_file,
            header=None, 
            index=None, 
            mode='a', 
            encoding='utf-8', 
            columns=['Labels', 'Text'],
            sep=' ',
            escapechar=' ',
            quoting=csv.QUOTE_NONE,
            quotechar=' '
           )

print('Ready ', test_file)


Ready  corpus10k.train
Ready  corpus10k.test


In [78]:
#-------------------------------------------------------
# Training the FastText model
#-------------------------------------------------------

print('FastText training Begin...', datetime.datetime.now())

# Command for training 

filename = os.path.splitext(file2open)[0]

input_file = filename + '.train'

output_file = filename

result_file = filename + ".result"

batcmd = './fasttext supervised -input '+ input_file + ' -output ' + \
                                          output_file + ' -dim 10 -lr 0.9 -wordNgrams 1 -minCount 1 -bucket 10000000 -epoch 25' + \
                                          '> ' + result_file

result = subprocess.check_output(batcmd, shell=True)

# print(result.decode('utf-8'))
# batcmd = 'cat ' + result_file
# result = subprocess.check_output(batcmd, shell=True)

# input_file = 'train-' + os.path.splitext(file2open)[0] + '.txt'

# output_file = 'train-' + os.path.splitext(file2open)[0]

# # ft_model = ft.train_supervised(input=input_file)

# !./fasttext supervised -input input_file -output output_file -dim 10 -lr 1 -wordNgrams 1 -minCount 1 -bucket 10000000 -epoch 25

# print('FastText Model Words\n')
# print(ft_model.words)

# print('FastText Model Labels\n')
# print(ft_model.labels)

print('FastText training End...', datetime.datetime.now())

print('Ready')


FastText training Begin... 2021-09-25 21:55:03.189536
FastText training End... 2021-09-25 21:55:08.285600
Ready


In [82]:
#-------------------------------------------------------
# Testing the fast text model
#-------------------------------------------------------

# Command for testing

input_file = filename + '.bin'

test_file  = filename + '.test'

# batcmd = 'fasttext test '+ input_file + ' ' + output_file + ' 2>$null'

batcmd = './fasttext test '+ input_file + ' ' + test_file 

print('fast text Begin...', datetime.datetime.now())

result = subprocess.check_output(batcmd, shell=True)

print('fast text End...', datetime.datetime.now())

print(result.decode('utf-8'))


fast text Begin... 2021-09-25 22:03:40.527759
fast text End... 2021-09-25 22:03:40.611738
N	1919
P@1	0.926
R@1	0.522



In [83]:
#-------------------------------------------------------
# Predicting labels
#-------------------------------------------------------

# File where question is written
question_file = open("question.txt", "w")

# File that contains the model
input_file = filename + '.bin'

# User is asked
question = input("Please, enter your question:\n")

# Question is cleaned and lemmatized
question2 = clean_txt(question)

# Question is written to a file
n = question_file.write(question2)

# Question file is closed
question_file.close()

# FastText string command is assembled
batcmd = './fasttext predict '+ input_file + ' question.txt 5 1>labels.csv'

# FastText commnd is executed. Output written to "labels.csv"
result = subprocess.check_output(batcmd, shell=True)

# labels are read and split into a list
csv_reader = csv.reader(codecs.open('labels.csv', 'rU', 'utf-8'))

lists_from_csv = []

for row in csv_reader:
    lists_from_csv.append(row)

list_from_csv = list(tuple(remove_label(l,'__label__') for l in lists_from_csv[0][0].split()))

print(list_from_csv)


Please, enter your question:
I want to write a program in Python that sends a message from a smartphone to the screens of several computers running different operating systems like Windows, Unix, MacOs, etc. across a TCP/IP network.
['networking', 'python', 'macos', 'windows', 'user-interface']


In [93]:
# change the current directory to specified directory
os.chdir(r"..")
 
print("Directory changed")

Directory changed


In [96]:
! rmdir result && ls -l

total 28472
drwxr-xr-x 2 root root     4096 Sep 25 18:01 alignment
-rw-r--r-- 1 root root    80592 Sep 25 21:10 args.o
-rw-r--r-- 1 root root   104976 Sep 25 21:10 autotune.o
drwxr-xr-x 5 root root     4096 Sep 25 18:38 build
-rwxr-xr-x 1 root root     1428 Sep 25 18:01 classification-example.sh
-rwxr-xr-x 1 root root     3154 Sep 25 18:01 classification-results.sh
-rw-r--r-- 1 root root     2214 Sep 25 18:01 CMakeLists.txt
-rw-r--r-- 1 root root     3356 Sep 25 18:01 CODE_OF_CONDUCT.md
-rw-r--r-- 1 root root     2061 Sep 25 18:01 CONTRIBUTING.md
-rw-r--r-- 1 root root  1859871 Sep 25 21:55 corpus10k.bin
-rw-r--r-- 1 root root 12393293 Sep 25 18:42 corpus10k.csv
-rw-r--r-- 1 root root        0 Sep 25 21:55 corpus10k.result
-rw-r--r-- 1 root root   700856 Sep 25 21:34 corpus10k.test
-rw-r--r-- 1 root root  6300834 Sep 25 21:34 corpus10k.train
-rw-r--r-- 1 root root  3033081 Sep 25 21:55 corpus10k.vec
drwxr-xr-x 2 root root     4096 Sep 25 18:01 crawl
drwxr-xr-x 3 root root     4096 Sep 